In [1]:
import tensorflow as tf
import math
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tqdm
from sklearn.model_selection import train_test_split

%matplotlib inline

/home/li/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with open('notMNIST.pickle', 'rb') as pfile:
    pickle_data = pickle.load(pfile)
    train_features = pickle_data['train_features']
    train_labels = pickle_data['train_labels']
    test_features = pickle_data['test_features']
    test_labels = pickle_data['test_labels']
    del pickle_data

In [3]:
features_count = 784
labels_count = 10

features = tf.placeholder(tf.float32)
labels = tf.placeholder(tf.float32)

weights = tf.Variable(tf.truncated_normal((features_count, labels_count)))
biases = tf.Variable(tf.zeros(labels_count))

In [4]:
train_features, valid_features, train_labels, valid_labels = train_test_split(train_features, 
                                                                             train_labels,
                                                                             test_size=0.05,
                                                                             random_state=10238)
print(train_features.shape)
print(valid_features.shape)

(199500, 784)
(10500, 784)


In [5]:
def train_nn(epochs, batch_size, learning_rate):

    logits = tf.matmul(features, weights) + biases
    prediction = tf.nn.softmax(logits)
    cross_entropy = -tf.reduce_sum(labels * tf.log(prediction), axis=1)
    loss = tf.reduce_mean(cross_entropy)
    is_correct_prediction = tf.equal(tf.argmax(labels, 1), tf.argmax(prediction, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct_prediction, tf.float32), axis=0)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    init = tf.global_variables_initializer()
    
    batch_count = int(math.ceil(train_features.shape[0]/batch_size))
    
    with tf.Session() as session:
        session.run(init)
        
        for epoch_i in range(epochs):
            
            for batch_i in range(batch_count):
                batch_start = batch_i * batch_size
                batch_features = train_features[batch_start:batch_start + batch_size]
                batch_labels = train_labels[batch_start:batch_start + batch_size]
                
                session.run(optimizer, feed_dict={features:batch_features, labels:batch_labels})
                
        test_acc = session.run(accuracy, feed_dict={features:test_features, labels:test_labels})
        test_loss = session.run(loss, feed_dict={features:test_features, labels:test_labels})
        
    return test_loss, test_acc, weights, biases


In [10]:
# epochs = [10, 20, 50, 80, 100]
# batch_size = [50, 100, 200, 300, 500, 1000, 2000, 4000]
# learning_rate = [0.01, 0.02, 0.05, 0.08, 0.1, 0.2, 0.3, 0.5, 0.8, 1]
# Good parameters: epochs = 50, batch_size = 100, learning_rate = 0.1 
epochs = [50]
batch_size = [100]
learning_rate = [0.1]
test_loss = {}
test_acc = {}
for ep in epochs:
    for bs in batch_size:
        for lr in learning_rate:
            test_loss[(ep, bs, lr)], test_acc[(ep, bs, lr)] = train_nn(ep, bs, lr)
            print('epochs = {}, batch_size = {}, learning_rate = {}, loss = {}, accuracy = {}'.format(
                ep, bs, lr, test_loss[(ep, bs, lr)], test_acc[(ep, bs, lr)]))

InternalError: Blas GEMM launch failed : a.shape=(100, 784), b.shape=(784, 10), m=100, n=10, k=784
	 [[Node: MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_Placeholder_0_0/_1, Variable/read)]]

Caused by op 'MatMul', defined at:
  File "/home/li/miniconda3/envs/carnd-term1/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/li/miniconda3/envs/carnd-term1/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/li/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/li/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/li/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()


In [6]:
test_loss, test_acc, weights, biases = train_nn(50, 100, 0.1)

In [8]:
print(test_loss, test_acc)

0.41360098 0.8908


In [22]:
with tf.Session() as session:
    w = weights.read_value()

In [26]:
print(w[0])

IndexError: too many indices for array